# Cutting stock problem

A factory received an order that requires cutting aluminum (Al) rods to specific sizes. The stock length of an Al rod is 5.600 meters.

The order is given below in terms of (length, quantities). Our job is to design the cutting pattern of the Al stock rod to minimize waste.

__Q1.1__ Find the optimal cutting solution. How many cutting patterns are used?

__Q1.2__ For the optimal cutting solution, how much material is wasted?

__Q1.3__ (Extra credit) How many unique (and valid) cutting patterns are there?

In [2]:
stock_length = 5.6
Al_pieces_order = [(1.380, 5),
            (1.520, 4),
            (1.710, 5),
            (1.820, 4),
            (1.930, 3),
            (2.000, 4),
            (2.220, 5)]

In [3]:
from pulp import LpMaximize, LpMinimize, LpStatus, lpSum, LpVariable, LpProblem
import pulp
solver_list=pulp.listSolvers(onlyAvailable=True)
print(solver_list)
import numpy as np

['PULP_CBC_CMD']


In [60]:
'''
listall=[]
for i in range(len(Al_pieces_order)):
    for j in range(Al_pieces_order[i][1]):
        listall.append(Al_pieces_order[i][0])
wasted_list=[]
while len(listall):
    print('LEN =', len(listall))
    x={i: LpVariable(name=f'n{i}', cat='Binary') for i in range(len(listall))}
    model=LpProblem(name="cutting_stock_problem", sense=LpMaximize)
    model+=lpSum(x[i]*listall[i] for i in range(len(listall))) <= stock_length
    model+=lpSum(x[i]*listall[i] for i in range(len(listall)))
    model.solve()
    cnt=0
    clear_index=[]
    for var in model.variables():
        if var.value()==1: 
            cnt+=listall[int(var.name[1:])]
            clear_index.append(int(var.name[1:]))
            print(f'{var.name} : {var.value()}')
    clear_index.sort()
    for c in reversed(clear_index):
        listall.pop(c)
    wasted_list.append(cnt)
cnt_wasted=0;
for i in wasted_list:
    cnt_wasted=5.6-i
print("Wasted = ", cnt_wasted)
print(wasted_list)
'''
pulp.listSolvers(onlyAvailable=True)

['PULP_CBC_CMD']

In [5]:
from itertools import combinations

def generate_combinations(cuts, stock_length):
    valid=set()
    for r in range(1, len(cuts)+1):
        subsets=combinations(cuts, r)
        for s in subsets:
            if sum(s) <= stock_length:
                valid.add(s)
    return valid

# Define the cuts max of one type is 5.6/that_type
cuts = []
for i in range(len(Al_pieces_order)):
    for j in range(int(stock_length/Al_pieces_order[i][0])):
        cuts.append(Al_pieces_order[i][0])

# Generate valid combinations of cuts
valid_patterns=generate_combinations(cuts, stock_length)

#for i in valid_combinations:
#    print(i)


valid_patterns=sorted(valid_patterns)
print('Patterns = ', len(valid_patterns))

Patterns =  91


In [19]:
# parameters
a=[]
for i in range(len(Al_pieces_order)):
    tmp=[]
    for pattern in valid_patterns:
        #print(pattern)
        cnt=0
        for j in pattern:
            if j==Al_pieces_order[i][0]:
                cnt+=1
        tmp.append(cnt)
    a.append(tmp)
    
wasted=[]
for combination in valid_patterns:
    wasted_value=0
    for c in combination: 
        wasted_value+=int(c*100)
    wasted.append((560-wasted_value))
print(wasted)
        
b=[]
for i in range(len(Al_pieces_order)):
    b.append(Al_pieces_order[i][1])

[422, 284, 146, 8, 132, 113, 102, 91, 84, 62, 270, 118, 99, 88, 77, 70, 48, 251, 80, 69, 58, 51, 29, 240, 58, 47, 40, 18, 229, 36, 29, 7, 222, 22, 0, 200, 408, 256, 104, 85, 74, 63, 56, 34, 237, 66, 55, 44, 37, 15, 226, 44, 33, 26, 4, 215, 22, 15, 208, 8, 186, 389, 218, 47, 36, 25, 18, 207, 25, 14, 7, 196, 3, 189, 167, 378, 196, 14, 3, 185, 178, 156, 367, 174, 167, 145, 360, 160, 138, 338, 116]


In [62]:
# https://neos-guide.org/case-studies/sc/mfg/the-cutting-stock-problem/
model=LpProblem(name="cutting_stock_problem", sense=LpMinimize)

In [63]:
#a={(i, j): LpVariable(name=f'{Al_pieces_order[i][0]}_in_p{j}', cat='Integer') for i in range(len(Al_pieces_order)) for j in range(len(valid_patterns))}
#b={i: LpVariable(name=f'{i}', cat='Integer') for i in range(len(Al_pieces_order))}
x={j: LpVariable(name=f'use_p{j}', lowBound=0, cat='Integer') for j in range(len(valid_patterns))}

In [64]:
# constraint: complete the order received
for i in range(len(Al_pieces_order)):
    model+=lpSum(a[i][j]*x[j] for j in range(len(valid_patterns))) == b[i]  # no wasted material 

## อ่อ wasted material คือไม่ให้เหลือเกินที่จะส่งให้ลูกค้า

In [65]:
# objective function

# minimize the used of stock
#model+=lpSum(x[j] for j in range(len(valid_patterns)))

# minimize wasted material
model+=lpSum(x[j]*wasted[j] for j in range(len(valid_patterns)))

In [66]:
status=model.solve()
status

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/panithi/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/5ddead230e3d4b659d04d04d64847aca-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/5ddead230e3d4b659d04d04d64847aca-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 473 RHS
At line 481 BOUNDS
At line 573 ENDATA
Problem MODEL has 7 rows, 91 columns and 188 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 60 - 0.00 seconds
Cgl0004I processed model has 7 rows, 91 columns (91 integer (9 of which binary)) and 188 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Full problem 7 rows 91 columns, reduced to 4 rows 24 columns
Cbc0012I Integer solution of 230 found by greedy equality after 0 iterations and 0 nodes (0.02 seconds)
Cbc0031I 1 added rows had average den

1

In [67]:
model.objective.value()/100

2.3

In [68]:
from decimal import Decimal
wasted_value=0
for var in model.variables():
    if var.value() > 0:
        print(f'{var.name} : {var.value()}')
        sum=0
        for i in valid_patterns[int(var.name[5:])]:
            sum+=int(i*100)
            print(int(i*100)/100, end=', ')
        #print('SUM =', sum)
        print('\nWASTED =', int(var.value())*(560-sum)/100)
        wasted_value+=int(var.value())*(560-sum)
        #print(sum(valid_patterns[int(var.name[5:])]))
        #print(valid_patterns[int(var.name[5:])])
        #wasted_value+=var.value()*(stock_length-sum(valid_patterns[int(var.name[5:])]))

print('Amount of wasted material is', wasted_value/100)

use_p3 : 1.0
1.38, 1.38, 1.38, 1.38, 
WASTED = 0.08
use_p31 : 1.0
1.38, 1.93, 2.22, 
WASTED = 0.07
use_p43 : 1.0
1.52, 1.52, 2.22, 
WASTED = 0.34
use_p49 : 1.0
1.52, 1.71, 2.22, 
WASTED = 0.15
use_p54 : 1.0
1.52, 1.82, 2.22, 
WASTED = 0.04
use_p70 : 3.0
1.71, 1.82, 2.0, 
WASTED = 0.21
use_p72 : 1.0
1.71, 1.93, 1.93, 
WASTED = 0.03
use_p88 : 1.0
2.0, 2.22, 
WASTED = 1.38
Amount of wasted material is 2.3
